# **CH01.2. Glove**

#### **`Paper Info`** : GloVe: Global Vectors for Word Representation
#### $ \hspace{1.75cm} - $ Jeffrey Pennington, Richard Socher, Christopher D. Manning et al. (EMNLP/2014)

<b></b>

> ## **요약(Summary)**

| Item | Description |
|------|------|
| Research topic | 전역적 단어 공기역(co-occurrence) 통계를 활용해 의미적·통사적 규칙성을 반영하는 단어 임베딩 학습 |
| Core idea | 단어 간 의미 관계를 공기역 확률 비율(log co-occurrence ratio)의 선형 구조로 모델링하고 이를 로그-쌍선형 회귀(log-bilinear regression)로 학습 |
| Key findings | $ \cdot{} $ 의미적 아날로지(analogy)에서 75% 정확도로 당시 SOTA 달성 <br> $ \cdot{} $ 유사도(similarity)와 개체명 인식(NER)에서 강력한 성능 <br> $ \cdot{} $ Skip-gram/CBOW 대비 빠른 수렴과 높은 효율성 |
| Contributions | $ \cdot{} $ 공기역 확률 비율에 기반한 임베딩 이론 제시 <br> $ \cdot{} $ Count-based + Predictive 모델의 통합적 관점 제안 <br> $ \cdot{} $ 가중 최소제곱(weighted least squares) 목적함수로 희소성 문제 해결 <br> $ \cdot{} $ 전역 통계 활용과 예측 기반 모델의 장점 결합 |

##### **(`PLUS`) 공기역(co-occurence)** : 말뭉치(corpus) 안에서 두 단어가 서로 근처에서 함께 등장하는 사건

<b></b>

> ## **연구 배경(Motivation & Prior Work)**

> ## **연구 배경(Motivation & Prior Work)**
#### **(1) 기존 연구 및 한계점** :
##### $ \hspace{0.15cm} \cdot{} $ LSA(Latent Semantic Analysis) 등의 행렬 분해(matrix factorization) 기반 방법은 전역 통계를 잘 반영하지만 아날로지(analogy) 관계 표현이 약함
##### $ \hspace{0.15cm} \cdot{} $ Skip-gram, CBOW와 같은 예측 기반(predictive) 모델은 의미적 선형 관계를 잘 포착하지만, 전역적 통계 활용이 비효율적
##### $ \hspace{0.15cm} \cdot{} $ Count-based 방법은 의미적 일반화 능력이 제한되고, Predictive 방법은 확률 정규화에 따른 연산 비용이 큼
##### $ \hspace{0.15cm} \cdot{} $ 따라서 두 접근의 장점을 결합하면서 의미적 선형성을 보존할 수 있는 새로운 모델이 요구됨

#### **(2) 연구 목표** :
##### $ \hspace{0.15cm} \cdot{} $ 단어 의미를 설명하는 핵심 요인이 공기역 확률의 비율(co-occurrence ratio)에 있음을 수학적으로 정식화
##### $ \hspace{0.15cm} \cdot{} $ 공기역 로그 확률을 선형 공간에서 근사하여 의미적 방향(linearity)을 유지하는 단어 벡터 학습
##### $ \hspace{0.15cm} \cdot{} $ Count-based 모델의 효율성과 Predictive 모델의 일반화 성능을 통합

#### **(3) 제안된 방법론** :
##### $ \hspace{0.15cm} \cdot{} $ 공기역 행렬의 로그 값을 쌍선형 회귀(log-bilinear regression)로 근사하는 가중 최소제곱 문제를 정의
##### $ \hspace{0.15cm} \cdot{} $ 공기역 확률 비율을 벡터 차이의 내적으로 표현함으로써 선형적 의미 관계를 모델링
##### $ \hspace{0.15cm} \cdot{} $ 가중 함수 $ f(x) $를 설계하여 희귀 및 초빈 단어의 비중을 조절

<b></b>

> ## **방법론(Method)**

#### **(1) 공기역 확률 비율(Co-occurrence Probability Ratio)** :
##### $ \hspace{0.15cm} \cdot{} $ 의미적 구별력은 절대적 확률 $ P(k|i) $보다 비율 $ \frac{P(k|i)}{P(k|j)} $에서 잘 드러남
##### $ \hspace{0.15cm} \cdot{} $ 예: $ k=\text{solid}$, $i=\text{ice}$, $j=\text{steam}$인 경우 비율이 매우 큼
##### $ \hspace{0.15cm} \cdot{} $ 이를 벡터 공간 상에서 함수 $ F $로 표현
##### $ \hspace{0.15cm} \Rightarrow{} F\big((w_{i}-w_{j})^{\top}\tilde{w}_{k}\big) = \frac{P_{ik}}{P_{jk}} \;\; \text{where } \, w_{i},\tilde{w}_{k}\in{}\mathbb{R}^{d} $

#### **(2) 로그 선형화(Log-linear Mapping)** :
##### $ \hspace{0.15cm} \cdot{} $ 덧셈 → 곱셈 관계를 보존하려면 $ F(x)=\exp{}(x) $ 여야 함 (군 동형성)
##### $ \hspace{0.15cm} \Rightarrow{} w_{i}^{\top}\tilde{w}_{k} = \log{}P_{ik} = \log{}X_{ik} - \log{}X_{i} $
##### $ \hspace{0.15cm} \cdot{} $ 상수항 $ -\log{}X_{i} $는 bias로 흡수
##### $ \hspace{0.15cm} \Rightarrow{} w_{i}^{\top}\tilde{w}_{j} + b_{i} + \tilde{b}_{j} = \log{}X_{ij} $

#### **(3) 손실 함수(Objective Function)** :
##### $ \hspace{0.15cm} \cdot{} $ 로그 공기역 값을 근사하는 가중 최소제곱(weighted least squares) 형태로 정의
##### $ \hspace{0.15cm} \Rightarrow{} J = \sum_{i,j} f(X_{ij}) \big(w_{i}^{\top}\tilde{w}_{j} + b_{i} + \tilde{b}_{j} - \log{}X_{ij}\big)^{2} $
##### $ \hspace{0.15cm} \text{where } \, f(X_{ij}) \text{ is weighting function controlling frequency impact} $

#### **(4) 가중 함수 설계(Weighting Function)** :
##### $ \hspace{0.15cm} \cdot{} $ 목적: 희귀 단어(노이즈) 억제, 초빈 단어 과도 반영 방지
##### $ \hspace{0.15cm} \Rightarrow{} f(x) = \begin{cases} (x/x_{\max})^{\alpha{}} & \text{if } x < x_{\max} \\ 1 & \text{otherwise} \end{cases} $
##### $ \hspace{0.15cm} \text{where } \, x_{\max}=100, \alpha{}=3/4 $

#### **(5) 기존 모델과의 관계(Relation to Other Models)** :
##### $ \hspace{0.15cm} \cdot{} $ Skip-gram 모델의 softmax 확률 $ Q_{ij} = \frac{\exp{}(w_{i}^{\top}\tilde{w}_{j})}{\sum_{k}\exp{}(w_{i}^{\top}\tilde{w}_{k})} $
##### $ \hspace{0.15cm} \cdot{} $ 이를 로그 공간에서 근사하면 GloVe의 최소제곱 형태와 유사
##### $ \hspace{0.15cm} \cdot{} $ 따라서 GloVe는 Count-based와 Predictive 접근의 중간적 형태로 해석 가능

<b></b>

> ## **실험(Experiments)**

#### **(1) 실험 설정** :
##### $ \hspace{0.15cm} \cdot{} $ 데이터셋
| Item | Description |
|------|--------------|
| Training dataset | Wikipedia 2010/2014, Gigaword 5, Common Crawl |
| Token size | 1B / 6B / 42B |
| Vocabulary size | 400K (Wiki+Giga), 2M (Common Crawl) |
| Evaluation dataset | Word Analogy, Word Similarity, NER(CoNLL-2003) |
| Evaluation size | Analogy (semantic/syntactic), 5 similarity benchmarks |
##### $ \hspace{0.15cm} \cdot{} $ 모델 및 훈련
| Item | Description |
|------|--------------|
| Model type | Log-bilinear regression (GloVe) |
| Embedding dimension | 50 / 100 / 200 / 300 |
| Context window size | Symmetric 10 (weighted by $1/d$) |
| Optimizer | AdaGrad (lr 0.05) |
| Epochs | 50 (<300D) / 100 (≥300D) |
| Training objective | Weighted least squares on log co-occurrence |
| Hardware | 32-core CPU (multi-threaded) |
| Runtime | ~14 min/iter (6B corpus, 300D) |
| Baseline models | Skip-gram, CBOW, SVD |
| Evaluation metric | Accuracy (Analogy), Spearman correlation (Similarity), F1 (NER) |

#### **(2) 분석 및 결과** :
##### $ \hspace{0.15cm} \cdot{} $ Analogy: 71.7% (6B), 75.0% (42B)로 당시 최고 성능
##### $ \hspace{0.15cm} \cdot{} $ Word Similarity: WS353(75.9), MC(83.6), RG(82.9) 등에서 높은 상관
##### $ \hspace{0.15cm} \cdot{} $ NER(CoNLL): GloVe 특징 추가 시 CRF F1=88.3%
##### $ \hspace{0.15cm} \cdot{} $ 벡터 차원(D)이 증가하면 성능 향상하나, 200~300D 이후 포화
##### $ \hspace{0.15cm} \cdot{} $ 작은 비대칭 윈도우는 통사(syntactic)에, 큰 대칭 윈도우는 의미(semantic)에 유리
##### $ \hspace{0.15cm} \cdot{} $ 대규모 코퍼스일수록 통사 성능은 지속 향상, 의미 성능은 도메인 영향 받음
##### $ \hspace{0.15cm} \cdot{} $ Skip-gram/CBOW 대비 빠른 수렴 및 더 우수한 시간-정확도 균형 확보

#### **(3) 결과 해석** :
##### $ \hspace{0.15cm} \cdot{} $ GloVe의 로그 회귀 구조는 확률 정규화가 필요 없어 학습 안정성이 높음
##### $ \hspace{0.15cm} \cdot{} $ 공기역 비율 기반 설계로 의미적 선형 관계(예: king - man ≈ queen - woman)가 자연히 형성
##### $ \hspace{0.15cm} \cdot{} $ Count-based 통계와 Predictive 학습 간의 통합적 시각을 제공

<b></b>

> ## **결론(Conclusion)**

#### **(1)** GloVe는 공기역 확률 비율의 로그 구조를 통해 의미적 선형성을 수학적으로 정당화한 단어 임베딩 모델임

#### **(2)** 전역 통계와 지역 예측의 장점을 결합하여 의미적·통사적 관계를 효과적으로 학습함

#### **(3)** 단순하고 효율적인 회귀 구조로 대규모 코퍼스에서도 빠른 수렴과 높은 일반화 성능을 보임

#### **(4)** 후속 연구(ELMo, BERT, GPT)의 기반이 된 ‘count + prediction hybrid’ 임베딩 패러다임을 정립함

##### **(`PLUS`) 한계 및 현대적 의의** :
##### $ \hspace{0.15cm} \cdot{} $ 문맥 비민감(static) 구조로 OOV와 의미 다의성 처리 한계 존재
##### $ \hspace{0.15cm} \cdot{} $ 그럼에도 불구하고 lightweight NLP task 및 학습 초기화(pretraining)에서 여전히 유용

<b></b>

> ## **부록(Appendix)**

<b></b>

> ## **비평(Commentary)**

<b></b>

> ## **참고 문헌(Reference)**

#### **(1) 관련 후속 논문** :
##### $ \hspace{0.15cm} \cdot{} $ 

#### **(2) 기타 참고 자료** :
#### - 블로그, 연구실 발표 자료 등등